In [1]:
import tensorflow
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.applications as ka
# from tensorflow.keras.keras_preprocessing.image import ImageDataGenerator
import numpy as np
import sys
# import sklearn.model_selection as skl
from sklearn.model_selection import train_test_split
import pickle

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=10,10

In [8]:
base_model = ka.ResNet50(
#                         include_top=True,
                        weights='imagenet',#,
#                         classes=2,
                        input_shape = (224, 224, 3)
                        )

# for layer in base_model.layers:
#     layer.trainable = False

# # # base_model_add_layers = keras.Model(inputs = base_model.input, outputs = base_model.output)
# # # out_1 = keras.layers.GlobalAvgPool2D()(base_model.output)
# # # out_1 = keras.layers.GlobalAveragePooling2D()(base_model.output)
# # x = keras.layers.Flatten()(base_model.layers[-2].output)
# # ae_fc_layer = keras.layers.Dense(2048, activation='relu')(base_model.layers[-3])
# # ae_conv1 = keras.layers.Conv2D(7, (1, 1), activation='relu')(tf.expand_dims(tf.expand_dims(ae_fc_layer, axis = -1), axis = -1))
ae_conv1 = keras.layers.Conv2D(512, (5, 5), padding = 'same', activation='relu')(base_model.layers[-3].output)
ae_up_1 = keras.layers.UpSampling2D((2, 2))(ae_conv1)
ae_conv2 = keras.layers.Conv2D(256, (5, 5), padding = 'same', activation='relu')(ae_up_1)
ae_up_2 = keras.layers.UpSampling2D((2, 2))(ae_conv2)
ae_conv3 = keras.layers.Conv2D(128, (5, 5), padding = 'same', activation='relu')(ae_up_2)
ae_up_3 = keras.layers.UpSampling2D((2, 2))(ae_conv3)
ae_conv4 = keras.layers.Conv2D(56, (5, 5), padding = 'same', activation='relu')(ae_up_3)
ae_up_4 = keras.layers.UpSampling2D((2, 2))(ae_conv4)
ae_conv5 = keras.layers.Conv2D(28, (5, 5), padding = 'same', activation='relu')(ae_up_4)
ae_up_5 = keras.layers.UpSampling2D((2, 2))(ae_conv5)
ae_conv6 = keras.layers.Conv2D(3, (5, 5), padding = 'same', activation='relu')(ae_up_5)

model = keras.Model(inputs = base_model.input, outputs = ae_conv6)#, base_model.output, base_model.layers[-2].output])
# # model_extra = keras.Model(inputs = model.input, outputs = [model.layers[-1].output, model.layers[-2].output, model.layers[-3].output])
# # # opt = keras.optimizers.SGD(lr = 0.0001)
opt = keras.optimizers.Adam(learning_rate = 0.0001, decay = 0.01)
# # # base_model.compile(optimizer = opt, loss = '_crossentropy', metrics = 'accuracy')
model.compile(optimizer = opt, loss = 'mse', metrics = 'accuracy')
# # model_extra.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = 'accuracy')

model.summary()
# base_model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
# directory = './ImageNet/organized_validation/'

# data_batches = tf.keras.preprocessing.image_dataset_from_directory(
#     directory,
#     labels="inferred",
#     label_mode="categorical",
# #     class_names=None,
#     color_mode="rgb",
#     batch_size=1,
#     image_size=(224, 224),
#     shuffle=True,
#     seed=13,
# #     validation_split=None,
# #     subset=None,
#     interpolation="bilinear"
# #     follow_links=False,
# )

In [ ]:
directory = './ImageNet/organized_validation_mini/'

resnet_preprocess = keras.applications.resnet.preprocess_input
# preprocessing_function = resnet_preprocess,
val_id_gen = keras.preprocessing.image.ImageDataGenerator(
                                                          validation_split = 0.25)
# val_gen = keras.preprocessing.image.ImageDataGenerator()
# val_id_gen = tensorflow.keras.preprocessing.image.ImageDataGenerator()
# val_gen = ImageDataGenerator()


train_gen = val_id_gen.flow_from_directory(
    directory, 
    target_size=(224, 224),
    color_mode='rgb', 
    classes=None,
    class_mode='categorical', 
    batch_size=1, 
    shuffle=True, 
    seed=13,
#     validation_split = 0.25,
    subset = 'training',
#     save_to_dir=None, 
#     save_prefix='', 
#     save_format='png', f
#     ollow_links=False,
#     subset=None, 
    interpolation='nearest'
)

val_gen = val_id_gen.flow_from_directory(
    directory, 
    target_size=(224, 224),
    color_mode='rgb', 
    classes=None,
    class_mode='categorical', 
    batch_size=1, 
    shuffle=True, 
    seed=13,
#     validation_split = 0.25,
    subset = 'validation',
#     save_to_dir=None, 
#     save_prefix='', 
#     save_format='png', f
#     ollow_links=False,
#     subset=None, 
    interpolation='nearest'
)

# next(val_gen)

In [ ]:
x_train = np.zeros((1140, 224, 224, 3))
x_val = np.zeros((360, 224, 224, 3))

for i in range(x_train.shape[0]):
    x_train[i, :, :, :] = train_gen.next()[0]
    
for j in range(x_val.shape[0]):
    x_val[j, :, :, :] = val_gen.next()[0]

In [ ]:
def scheduleFunc():
    

tf.keras.callbacks.LearningRateScheduler(scheduleFunc, verbose=0)

In [ ]:
# Train model on generator:

history = model.fit(x_train, x_train, validation_data = (x_val, x_val), epochs = 10, shuffle = True, workers = 4, batch_size = 32)


In [ ]:
# Redefine generator for encodings:

In [ ]:
x_val = np.zeros((10000, 224, 224, 3))
y_val = np.zeros((10000, 1000))

# x_encoded = np.zeros((50000, 2048))
# y_encoded = np.zeros((50000, 1000))

In [ ]:
for z in range(1, 6):
    for i in range(10000):
        if i % 1000 == 0:
            print(i)

        x, y = val_gen.next()
        x_val[i, :, :, :] = x
        y_val[i, :] = y

    # x_train, x_val, y_train, y_val = train_test_split(x_val, y_val, train_size = .75, shuffle = True)

#     out = model_extra.predict(x_val)
    out = base_model.predict(x_val)
    
#     print(a.shape)
#     print(b.shape)
#     print(c.shape)

    print(out.shape)

    
    pickle.dump( out, open( "imagenet_val_1000_encoding_" + str(z) + ".p", "wb" ) )


#     x_encoded[z:z+10000, :] = c
#     y_encoded[z:z+10000, :] = y_val
    

In [ ]:
print(out.shape)

for pred in out:
#     print(pred.shape)
    pred = pred.reshape(1, 1000)
    decoded = keras.applications.resnet.decode_predictions(
        pred, top=5
    )
    print(decoded)

In [ ]:
print(out_intermediate[2].shape)

In [ ]:
out = out_intermediate

dist_from_first = []
image_dict = {}
index = 1444
first_features = out[2][index]
first_image = x_val[index][:,:,:]
print(first_image.shape)

# hash images:
for idx in range(x_val.shape[0]):
    image_dict[str(out[2][idx])] = x_val[idx] 

plt.clf()
plt.imshow(first_image/255)
plt.show()

# for i in range(400):
#     if i % 25 == 0:
#         print(i)
for idx, item in enumerate(out[2]):
    dist_from_first.append((np.linalg.norm(first_features-item), (str(item))))

dist_from_first.sort()
# print(dist_from_first)

for i in range(20):
    plt.clf()
    img = image_dict[dist_from_first[i][1]]
    plt.imshow(img/255)
    plt.show()

In [ ]:
for i in range(10):
    if i % 1000 == 0:
        print(i)

    x, y = val_gen.next()
#     x_val[i, :, :, :] = x
#     y_val[i, :] = y
    
    out = model.predict(x)
    
    plt.clf()
    plt.imshow(x[0]/255)
    plt.show()
    
    plt.clf()
    plt.imshow(out[0]/255)
    plt.show()


10000000000.0